In [ ]:
import hashlib
import random
import json
import string
import binascii
import numpy as np
import pandas as pd
import pylab as pl
import logging
%matplotlib inline

In [ ]:
def sha256(message):
    return hashlib.sha256(message.encode('ascii')).hexdigest()

In [ ]:
message = 'hello world'
for nonce in range(1000):
    digest = sha256(message + str(nonce))
    if digest.startswith('11'):
        print('found nonce = %d' % nonce)
        break
print(sha256(message + str(nonce)))

In [ ]:
def block_hash(message):
    return sha256(message)

In [ ]:
def mine(message, difficulty=1):
    assert difficulty >=1
    i=0
    prefix = '1' * difficulty
    while True:
        nonce = str(i)
        digest = block_hash(message+nonce)
        if digest.startswith(prefix):
            return nonce, i
        i += 1

In [ ]:
nonce, nitters = mine('hello welp', difficulty=3)
print('took %d iteration' %nitters)

In [ ]:
# %pip install pycryptodome
import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5


In [ ]:
class wallet(object):
    def __init__(self):
        random_gen = Crypto.Random.new().read
        self._private_key = RSA.generate(1024, random_gen)
        self._public_key = self._private_key.public_key()
        self._signer = PKCS1_v1_5.new(self._private_key)
    
    @property
    def address(self):
        return binascii.hexlify(self._public_key.export_key(format='DER')).decode('ascii')
    
    def sign(self, message):
        h = SHA.new(message.encode('utf8'))
        return binascii.hexlify(self._signer.sign(h)).decode('ascii')
    
def verifySignature(wallet_address, message, signature):
    pubkey = RSA.import_key(binascii.unhexlify(wallet_address))
    verifier = PKCS1_v1_5.new(pubkey)
    h = SHA.new(message.encode('utf8'))
    return verifier.verify(h, binascii.unhexlify(signature))


In [ ]:
w1 = wallet()
signature = w1.sign('foobar')
assert verifySignature(w1.address, 'foobar', signature)
assert not verifySignature(w1.address, 'rogue message', signature)

In [ ]:
class TransactionInput(object):
    def __init__(self, transaction, output_index):
        self.transaction = transaction
        self.output_index = output_index
        assert 0 <= self.output_index < len(transaction.outputs)
    
    def to_dict(self):
        d = {
            'transaction' : self.transaction.hash(),
            'output_index' : self.output_index
        }
        return d
    
    @property
    def parent_output(self):
        return self.transaction.outputs[self.output_index]

In [ ]:
class TransactionOutput(object):
    def __init__(self, recipient_address, amount):
        self.recipient = recipient_address 
        self.amount = amount
    
    def to_dict(self):
        d = {
            'recipient_address' : self.recipient, 
            'amount' : self.amount
        }
        return d
    

In [ ]:
def computeFee(inputs, outputs):
    total_in = sum(i.transaction.outputs[i.output_index].amount for i in inputs)
    total_out = sum(o.amount for o in outputs)  
    assert total_out <= total_in, "Invalid transaction"
    return total_in - total_out

In [ ]:
class Transaction(object):
    def __init__(self, wallet, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs
        self.fee = computeFee(inputs, outputs)
        self.signature = wallet.sign(json.dumps(self.to_dict(include_signature=False)))
    
    def to_dict(self, include_signature=True):
        d={
            "inputs" : list(map(TransactionInput.to_dict, self.inputs)),
            "outpts" :  list(map(TransactionOutput.to_dict, self.outputs)),
            "fee"    :  self.fee
        }
        if include_signature:
            d["signature"] = self.signature
        return d
    
    def hash(self):
        return block_hash(json.dumps(self.to_dict(include_signature=False)))
    

In [ ]:
def to_dict(self, include_signature=True):
    d={
        "inputs" : list(map(TransactionInput.to_dict, self.inputs)),
        "outputs" :  list(map(TransactionOutput.to_dict, self.outputs)), 
        "fee"    :  self.fee
    }
    if include_signature:
        d["signature"] = self.signature
    return d

In [ ]:
def to_dict(self, include_signature=True):
    d={
        "inputs" : list(map(TransactionInput.to_dict, self.inputs)),
        "outputs" :  list(map(TransactionOutput.to_dict, self.outputs)),  # Fixed typo: "outpts" -> "outputs"
        "fee"    :  self.fee
    }
    if include_signature:
        d["signature"] = self.signature
    return d

In [ ]:
class GenesisTransaction(Transaction):
    def __init__(self, recipient_address, amount=25):
        self.inputs = []
        self.outputs = [
            TransactionOutput(recipient_address, amount)
        ]
        self.fee = 0
        self.signature = "genesis"
    
    def to_dict(self, include_signature=True):
        assert not include_signature, "Cannot include signature of genesis transaction"
        return super().to_dict(include_signature=False)

In [ ]:
alice = wallet()
bob = wallet()

t1 = GenesisTransaction(alice.address)
t2 = Transaction(
    alice,
    [TransactionInput(t1, 0)],
    [TransactionOutput(bob.address, 2.0), TransactionOutput(alice.address, 22.0)]
)
assert np.abs(t2.fee - 1.0) < 1e-5

In [ ]:
alice = wallet()
bob = wallet()

t1 = GenesisTransaction(alice.address)
t2 = Transaction(
    alice,
    [TransactionInput(t1, 0)],
    [TransactionOutput(bob.address, 2.0), TransactionOutput(alice.address, 22.0)]
)
assert np.abs(t2.fee - 1.0) < 1e-5

In [ ]:
alice = wallet()
bob = wallet()
walter = wallet()

# This gives 25 coins to Alice
t1 = GenesisTransaction(alice.address)

# Of those 25, Alice will spend
# Alice -- 5 --> Bob
#       -- 15 --> Alice
#       -- 5 --> Walter
t2 = Transaction(
    alice,
    [TransactionInput(t1, 0)],
    [TransactionOutput(bob.address, 5.0), TransactionOutput(alice.address, 15.0), TransactionOutput(walter.address, 5.0)]
)

# Walter -- 5 --> Bob
t3 = Transaction(
    walter,
    [TransactionInput(t2, 2)],
    [TransactionOutput(bob.address, 5.0)])

# Bob -- 8 --> Walter
#     -- 1 --> Bob
#        1 fee
t4 = Transaction(
    bob,
    [TransactionInput(t2, 0), TransactionInput(t3, 0)],
    [TransactionOutput(walter.address, 8.0), TransactionOutput(bob.address, 1.0)]
)

transactions = [t1, t2, t3, t4]



In [ ]:
def computeBalance(wallet_address, transactions):
    balance = 0
    for t in transactions:
        for txin in t.inputs:
            if txin.parent_output.recipient == wallet_address:
                balance -= txin.parent_output.amount
        
        for txout in t.outputs:
            if txout.recipient == wallet_address:
                balance += txout.amount

    return balance

print("Alice  has %.02f anchorcoins" % computeBalance(alice.address, transactions))
print("Bob    has %.02f anchorcoins" % computeBalance(bob.address, transactions))
print("Walter has %.02f anchorcoins" % computeBalance(walter.address, transactions))

In [ ]:
def verifyTransaction(transaction):
    tx_message = json.dumps(transaction.to_dict(include_signature = False))
    if isinstance(transaction, GenesisTransaction):
        return True
    
    for tx in transaction.inputs:
        if not verifyTransaction(tx.transaction):   
            logging.error("Invalid parent transaction")
            return False
    
    first_input_address = transaction.inputs[0].parent_output.recipient
    for txin in transaction.inputs[1:]:
        if txin.parent_output.recipient != first_input_address:
            logging.error(
                "Transaction inputs belong to multiple wallets (%s and %s)" %
                (txin.parent_output.recipient, first_input_address)
            )
            return False
    
    if not verifySignature(first_input_address, tx_message, transaction.signature):
        logging.error("Invalid transaction signature, trying to spend someone else's money ?")
        return False
    
    computeFee(transaction.inputs, transaction.outputs)
    return True    

In [ ]:
t1 = GenesisTransaction(alice.address)
assert verifyTransaction(t1)

In [ ]:
BLOCK_INCENTIVE = 25
DIFFICULTY = 2

def compute_total_fee(transactions):
    return sum(t.fee for t in transactions)

class Block(object):
    def __init__(self, transactions, ancestor, miner_address, skip_verify=False):
        reward = compute_total_fee(transactions) + BLOCK_INCENTIVE
        self.transactions = [GenesisTransaction(miner_address, amount=reward)] + transactions
        self.ancestor = ancestor

        if not skip_verify:
            assert all(map(verifyTransaction, transactions))
        
        json_block = json.dumps(self.to_dict(include_hash = False))
        self.nonce, _ = mine(json_block, DIFFICULTY)
        self.hash = block_hash(json_block + self.nonce)
    
    def fee(self):
        return compute_total_fee(self.transactions)
    
    def to_dict(self, include_hash=False):
        d = {
            "transactions" : list(map(Transaction.to_dict, self.transactions)),
            "previous_block" : self.ancestor.hash
        }
        if include_hash:
            d["nonce"] = self.nonce
            d["hash"] = self.hash
        return d
    
class GenesisBlock(Block):       
    def __init__(self, miner_address):
        super(GenesisBlock, self).__init__(transactions=[], ancestor=None, miner_address=miner_address)
        
    def to_dict(self, include_hash=True):
        d = {
            "transactions" : [],
            "genesis_block" : True,
        }
        if include_hash == True:
            d["nonce"] = self.nonce
            d["hash"] = self.hash
        return d

In [ ]:
def verify_block(Block ,genesisBlock, used_output=None):
    if used_output is None:
        used_output = set()
    prefix = '1' * DIFFICULTY
    if not Block.hash.startswith(prefix):
        logging.error("Block hash (%s) doesn't start with prefix %s" % (Block.hash, prefix))
        return False
    if not all(map(verifyTransaction, Block.transactions)):
        return False
    
    for transactions in Block.transactions:
        for i in transactions.inputs:
            if i.parent_output in used_output:
                logging.error("Transaction uses an already spent output : %s" % json.dumps(i.parent_output.to_dict()))
                return False
            used_output.add(i.parent_output)
    
    if not (Block.hash == genesisBlock.hash):
        if not verify_block(Block.ancestor, genesisBlock, used_output):
            logging.error("Failed to validate ancestor block")
            return False
        
    tx0 = Block.transactions[0]
    if not isinstance(tx0, GenesisTransaction):
        logging.error("Transaction 0 is not a GenesisTransaction")
        return False
    if not len(tx0.outputs) == 1:
        logging.error("Transactions 0 doesn't have exactly 1 output")
        return False
    reward = compute_total_fee(Block.transactions[1:]) + BLOCK_INCENTIVE
    if not tx0.outputs[0].amount == reward:
        logging.error("Invalid amount in transaction 0 : %d, expected %d" % (tx0.outputs[0].amount, reward))
        return False
    
    for i, tx in enumerate(Block.transactions):
        if i == 0:
            if not isinstance(tx, GenesisTransaction):
                logging.error("Non-genesis transaction at index 0")
                return False  
        elif isinstance(tx, GenesisTransaction):
            logging.error("GenesisTransaction (hash=%s) at index %d != 0", tx.hash(), i)
            return False
    return True

In [ ]:
alice = wallet()
bob = wallet()
walter = wallet()

genesis_block = GenesisBlock(miner_address=alice.address)
print("genesis_block : " + genesis_block.hash + " with fee=" + str(genesis_block.fee()))

t1 = genesis_block.transactions[0]
t2 = Transaction(
    alice,
    [TransactionInput(t1, 0)],
    [TransactionOutput(bob.address, 5.0), TransactionOutput(alice.address, 15.0), TransactionOutput(walter.address, 5.0)]
)
t3 = Transaction(
    walter,
    [TransactionInput(t2, 2)],
    [TransactionOutput(bob.address, 5.0)])

t4 = Transaction(
    bob,
    [TransactionInput(t2, 0), TransactionInput(t3, 0)],
    [TransactionOutput(walter.address, 8.0), TransactionOutput(bob.address, 1.0)]
)

block1 = Block([t2], ancestor=genesis_block, miner_address=walter.address)
print("block1        : " + block1.hash + " with fee=" + str(block1.fee()))

block2 = Block([t3, t4], ancestor=block1, miner_address=walter.address)
print("block2        : " + block2.hash + " with fee=" + str(block2.fee()))

In [ ]:
verify_block(block1,genesis_block)
verify_block(block2, genesis_block)

In [ ]:
def collectTransaction(block, genesis_block):
    transactions = [] + block.transactions
    if block.hash != genesis_block.hash:
        transactions += collectTransaction(block.ancestor, genesis_block)
    return transactions

transactions = collectTransaction(block2, genesis_block)

# Alice mined 25 (from the genesis block) and gave 5 to bob and 5 to walter
print("Alice  has %.02f anchorcoin" % computeBalance(alice.address, transactions))
# Bob received 5 from alice and 5 from walter, but then back 8 to walter with a transaction fee of 1
print("Bob    has %.02f anchorcoin" % computeBalance(bob.address, transactions))
# Walter mined 2 blocks (2 * 25), received 8 from bob and go a transaction fee of 1 on block2
print("Walter has %.02f anchorcoin" % computeBalance(walter.address, transactions))